In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv('data.csv', encoding = 'ISO-8859-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom


In [6]:
def find_best_selling_products(dataframe, n=10):
    products = dataframe.groupby('Description')['Quantity'].sum(numeric_only=False)
    products = products.sort_values(ascending=False)
    return products.head(n)

In [7]:
find_best_selling_products(df)

Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     53847
JUMBO BAG RED RETROSPOT               47363
ASSORTED COLOUR BIRD ORNAMENT         36381
POPCORN HOLDER                        36334
PACK OF 72 RETROSPOT CAKE CASES       36039
WHITE HANGING HEART T-LIGHT HOLDER    35317
RABBIT NIGHT LIGHT                    31012
MINI PAINT SET VINTAGE                26437
PACK OF 12 LONDON TISSUES             26315
PACK OF 60 PINK PAISLEY CAKE CASES    24753
Name: Quantity, dtype: int64

In [8]:
def find_most_returned_products(dataframe, n=10):
    products = dataframe[dataframe['Quantity'] < 0].groupby('Description')['Quantity'].sum(numeric_only=False)
    products = products.sort_values(ascending=True)
    return products.head(n)

In [9]:
find_most_returned_products(df)

Description
PAPER CRAFT , LITTLE BIRDIE           -80995
MEDIUM CERAMIC TOP STORAGE JAR        -74494
ROTATING SILVER ANGELS T-LIGHT HLDR    -9376
POPART WOODEN PENCILS ASST             -3667
PIECE OF CAMO STATIONERY SET           -3275
FAIRY CAKE FLANNEL ASSORTED COLOUR     -3150
PINK BLUE FELT CRAFT TRINKET BOX       -2617
WHITE HANGING HEART T-LIGHT HOLDER     -2578
GIN + TONIC DIET METAL SIGN            -2048
HERB MARKER BASIL                      -1527
Name: Quantity, dtype: int64

In [10]:
def find_best_customers(dataframe, n=10):
    customers = dataframe.groupby('CustomerID')['Quantity'].sum(numeric_only=False)
    customers = customers.sort_values(ascending=False)
    return customers.head(n)

In [11]:
find_best_customers(df)

CustomerID
-1        405327
 14646    197200
 12415     77242
 14911     76987
 17450     69011
 18102     64124
 17511     62753
 13694     61669
 14298     57919
 14156     56958
Name: Quantity, dtype: int64

In [12]:
def find_most_returned_customers(dataframe, n=10):
    customers = dataframe[dataframe['Quantity'] < 0].groupby('CustomerID')['Quantity'].sum(numeric_only=False)
    customers = customers.sort_values(ascending=True)
    return customers.head(n)

In [13]:
find_most_returned_customers(df)

CustomerID
 16446   -80995
 12346   -74215
-1       -46667
 15838    -9360
 15749    -9014
 16029    -8005
 12931    -4427
 14607    -3768
 14911    -3332
 17949    -2878
Name: Quantity, dtype: int64

In [14]:
def find_best_selling_products_by_country(dataframe, n=10):
    # Only get rows where quantity is greater than zero (to ignore returns)
    df = dataframe[dataframe['Quantity'] > 0]
    countries_products = df.groupby(['Country','Description'])[['Quantity']].sum(numeric_only=False).reset_index()
    countries_products = countries_products.sort_values(by=['Country','Quantity'],ascending=[True,False])
    products_by_country = {}
    for country, df_country in countries_products.groupby('Country'):
        products = df_country.head(n)["Description"].tolist()
        products_by_country[country] = products
    return products_by_country

In [15]:
find_best_selling_products_by_country(df)

{'Australia': ['MINI PAINT SET VINTAGE ',
  'RABBIT NIGHT LIGHT',
  'RED  HARMONICA IN BOX ',
  'RED TOADSTOOL LED NIGHT LIGHT',
  'HOMEMADE JAM SCENTED CANDLES',
  'DOLLY GIRL LUNCH BOX',
  'SPACEBOY LUNCH BOX ',
  'BLUE HARMONICA IN BOX ',
  'MINI JIGSAW SPACEBOY',
  '4 TRADITIONAL SPINNING TOPS'],
 'Austria': ['SET 12 KIDS COLOUR  CHALK STICKS',
  'MINI JIGSAW PURDEY',
  'PACK OF 6 PANNETONE GIFT BOXES',
  'PACK OF 6 SWEETIE GIFT BOXES',
  '12 PENCILS SMALL TUBE RED RETROSPOT',
  'ANGEL DECORATION 3 BUTTONS ',
  'KIDS RAIN MAC BLUE',
  'KIDS RAIN MAC PINK',
  'PACK OF 60 PINK PAISLEY CAKE CASES',
  'PARTY CONES CANDY ASSORTED'],
 'Bahrain': ['ICE CREAM SUNDAE LIP GLOSS',
  'DOUGHNUT LIP GLOSS ',
  'OCEAN SCENT CANDLE IN JEWELLED BOX',
  'GROW A FLYTRAP OR SUNFLOWER IN TIN',
  'S/4 PINK FLOWER CANDLES IN BOWL',
  'MINI CAKE STAND WITH HANGING CAKES',
  'STRAWBERRY FAIRY CAKE TEAPOT',
  'CERAMIC CAKE BOWL + HANGING CAKES',
  'GREEN REGENCY TEACUP AND SAUCER',
  'NOVELTY BISCUITS CAKE 

In [16]:
def find_similar_products_countries(dataframe, n=10):
    # Create a dataframe with the top n products in the United Kingdom
    df_uk = dataframe[dataframe['Country'] == 'United Kingdom']
    uk_products = df_uk.groupby('Description')[['Quantity']].sum(numeric_only=False).nlargest(n, 'Quantity')
    uk_products = uk_products.index.tolist()

    # Group the original dataframe by country and get the top n products for each country
    countries_products = dataframe.groupby(['Country','Description'])[['Quantity']].sum(numeric_only=False).reset_index()
    countries_products = countries_products.sort_values(by=['Country','Quantity'],ascending=[True,False])
    countries_products = countries_products.groupby('Country').head(n)
    similar_countries = {}

    # Iterate over the countries and compare the top n products to the UK products
    for country, df_country in countries_products.groupby('Country'):
        country_products = df_country['Description'].tolist()
        common_products = set(country_products).intersection(uk_products)
        if len(common_products) > 0:
            similar_countries[country] = common_products
    return similar_countries

In [17]:
find_similar_products_countries(df)

{'Austria': {'PACK OF 60 PINK PAISLEY CAKE CASES'},
 'Belgium': {'PACK OF 72 RETROSPOT CAKE CASES'},
 'Canada': {'WORLD WAR 2 GLIDERS ASSTD DESIGNS'},
 'Channel Islands': {'JUMBO BAG RED RETROSPOT',
  'PACK OF 72 RETROSPOT CAKE CASES',
  'WHITE HANGING HEART T-LIGHT HOLDER'},
 'Cyprus': {'WHITE HANGING HEART T-LIGHT HOLDER'},
 'Denmark': {'WORLD WAR 2 GLIDERS ASSTD DESIGNS'},
 'EIRE': {'PACK OF 60 PINK PAISLEY CAKE CASES',
  'PACK OF 72 RETROSPOT CAKE CASES'},
 'Finland': {'PACK OF 60 PINK PAISLEY CAKE CASES'},
 'France': {'ASSORTED COLOUR BIRD ORNAMENT',
  'PACK OF 72 RETROSPOT CAKE CASES'},
 'Germany': {'PACK OF 72 RETROSPOT CAKE CASES'},
 'Hong Kong': {'WORLD WAR 2 GLIDERS ASSTD DESIGNS'},
 'Italy': {'POPCORN HOLDER'},
 'Japan': {'WORLD WAR 2 GLIDERS ASSTD DESIGNS'},
 'Netherlands': {'JUMBO BAG RED RETROSPOT', 'PACK OF 72 RETROSPOT CAKE CASES'},
 'Norway': {'PACK OF 60 PINK PAISLEY CAKE CASES',
  'PACK OF 72 RETROSPOT CAKE CASES',
  'POPCORN HOLDER'},
 'Portugal': {'PACK OF 72 RETRO

In [22]:
def find_product_with_biggest_variation(df, start_date, end_date, start_date2, end_date2, number = 10, ascending = False):
            start_date = pd.to_datetime(start_date)
            end_date = pd.to_datetime(end_date)
            start_date2 = pd.to_datetime(start_date2)
            end_date2 = pd.to_datetime(end_date2)
            # Filter the dataframe to only include the products that were sold in the given time period
            df1 = df[(pd.to_datetime(df['InvoiceDate']) >= start_date) & (pd.to_datetime(df['InvoiceDate']) <= end_date)]
            df2 = df[(pd.to_datetime(df['InvoiceDate']) >= start_date2) & (pd.to_datetime(df['InvoiceDate']) <= end_date2)]
            df1 = df1[df1['Quantity'] > 0]
            df2 = df2[df2['Quantity'] > 0]
            df1 = df1.groupby('Description')[['Quantity']].sum(numeric_only=False).reset_index()
            # print  GLITTER CHRISTMAS HEART	
            print(df1[df1['Description'] == 'FELT FARM ANIMAL WHITE BUNNY '])    


            df2 = df2.groupby('Description')[['Quantity']].sum(numeric_only=False).reset_index()
            print(df2[df2['Description'] == 'FELT FARM ANIMAL WHITE BUNNY '])    

            # calculate total quantity sold for each product
            df1 = df1.sort_values(by=['Quantity'],ascending=ascending)
            df2 = df2.sort_values(by=['Quantity'],ascending=ascending)
            # calculate the variation between the two time periods for each product in percentage
            df = pd.merge(df1, df2, on='Description', how='outer')
            df['Variation'] = (df['Quantity_y'] - df['Quantity_x']) / df['Quantity_x'] * 100
            df = df.sort_values(by=['Variation'],ascending=ascending)
            df = df.dropna()
            # delete the columns that are not needed , Quantity_x and Quantity_y
            df = df.drop(['Quantity_x', 'Quantity_y'], axis=1)
            # sort the dataframe by variation in ascending order
            df = df.sort_values(by=['Variation'],ascending=ascending)
            # return as a dict 
            dict = {}
            for _, row in df.iterrows():
                dict[row['Description']] = row['Variation']
                if len(dict) == number:
                    break
            return dict



In [23]:
find_product_with_biggest_variation(df, '2010-12-01', '2010-12-31', '2011-12-01', '2011-12-31', 10, True)

                       Description  Quantity
883  FELT FARM ANIMAL WHITE BUNNY        543
                       Description  Quantity
698  FELT FARM ANIMAL WHITE BUNNY          1


{'FELT FARM ANIMAL WHITE BUNNY ': -99.8158379373849,
 'JUMBO BAG TOYS ': -99.65870307167235,
 'HANGING FAIRY CAKE DECORATION': -99.65277777777779,
 'METAL SIGN CUPCAKE SINGLE HOOK': -99.52830188679245,
 'RED HEARTS LIGHT CHAIN ': -99.31972789115646,
 'ASSTD COL BUTTERFLY/CRYSTAL W/CHIME': -99.3103448275862,
 'METAL SIGN,CUPCAKE SINGLE HOOK': -99.14040114613181,
 'RED RETROSPOT MUG': -99.11111111111111,
 'BIRTHDAY PARTY CORDON BARRIER TAPE': -99.09909909909909,
 'BLACK LOVE BIRD T-LIGHT HOLDER': -99.09909909909909}